# Get Online Inference from Vertex AI Endpoint - Example Template

This notebook demonstrates how to get predictions from a deployed Vertex AI Endpoint.
**This is an example template - copy and configure for your use.**

## Setup Instructions:

1. Copy this notebook and remove `.example` from the filename
2. Set up your configuration using one of the methods below
3. Run the notebook cells


## Configuration Options

### Option 1: Use .env file (Recommended)

1. Copy `.env.example` to `.env` in **this directory** (06_Get_Online_Inference/)
2. Edit `.env` with your GCP project details
3. Run the config cell below - it will load from .env automatically

### Option 2: Direct Configuration

Uncomment and fill in values directly in the configuration cell


In [ ]:
# Import Required Libraries
from google.cloud import aiplatform
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

print("Libraries imported successfully.")

## Configuration Method 1: Using Local config.py (Recommended)

**Setup:**

1. Copy `.env.example` to `.env` in this directory3. The config.py file will automatically load from .env
2. Edit `.env` with your GCP project details


In [ ]:
# Option 1: Load from local config.py
try:
    from config import GCP_PROJECT_ID, GCP_REGION, ENDPOINT_DISPLAY_NAME

    PROJECT_ID = GCP_PROJECT_ID
    REGION = GCP_REGION

    print("✅ Configuration loaded from config.py")
    print(f"Project: {PROJECT_ID}")
    print(f"Region: {REGION}")
    print(f"Endpoint: {ENDPOINT_DISPLAY_NAME}")
except ImportError:
    print("⚠️ config.py not found in this directory.")
    print("Please create .env file (copy from .env.example) or use Option 2 below.")
    PROJECT_ID = None

    REGION = None    ENDPOINT_DISPLAY_NAME = None

## Configuration Method 2: Direct Configuration

Uncomment and fill in if not using config.py:


In [ ]:
# Option 2: Direct configuration (uncomment and fill in)
# PROJECT_ID = "your-project-id-here"
# REGION = "us-central1"
# ENDPOINT_DISPLAY_NAME = "your-endpoint-name"

# Verify configuration
if PROJECT_ID:
    print(f"Project: {PROJECT_ID}")
    print(f"Region: {REGION}")
    print(f"Endpoint Name: {ENDPOINT_DISPLAY_NAME}")
else:
    print("❌ Please configure PROJECT_ID, REGION, and ENDPOINT_DISPLAY_NAME")

## Train Model and Create Joblib Files

Before getting online inference, we need to train the model and save it as joblib files.
**Note:** Replace the CSV path with your actual data file path.


In [ ]:
# Load the women's clothing e-commerce reviews dataset
# TODO: Update path to your data file
df = pd.read_csv("../womens_clothing_ecommerce_reviews.csv")

print("Dataset loaded successfully.")
print(f"Dataset shape: {df.shape}")
print(df.head())

In [ ]:
# Define features (X) and target variable (Y)
X = df['Review Text']
Y = df['sentiment']

# Verify data types
print(type(X))
print(type(Y))

In [ ]:
# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

print("\nShapes of Train/Test Datasets:")
print("X_train shape:", x_train.shape)
print("Y_train shape:", y_train.shape)
print("X_test shape:", x_test.shape)
print("Y_test shape:", y_test.shape)

In [ ]:
# Initialize and train CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
x_train_bow = vectorizer.fit_transform(x_train)
x_test_bow = vectorizer.transform(x_test)

print("\nText vectorization completed.")
print("Training set shape:", x_train_bow.shape)
print("Testing set shape:", x_test_bow.shape)

In [ ]:
# Train the Logistic Regression model
model = LogisticRegression(max_iter=5000, random_state=42, solver='saga')
model.fit(x_train_bow, y_train)

print("\nModel Training Completed.")

In [ ]:
# Make predictions and evaluate the model
y_pred = model.predict(x_test_bow)
accuracy = accuracy_score(y_test, y_pred)

print("\nModel Accuracy on Test Set: {:.2f}%".format(accuracy * 100))

In [ ]:
# Save the model and vectorizer as joblib files
joblib.dump(model, 'model.joblib')
joblib.dump(vectorizer, 'vectorizer.joblib')

print("\n✅ Model saved as 'model.joblib'")
print("✅ Vectorizer saved as 'vectorizer.joblib'")
print("\nThese files can now be used for online inference.")

## Initialize Vertex AI and Get Endpoint


In [ ]:
# Initialize Vertex AI SDK
if PROJECT_ID and PROJECT_ID != "YOUR_PROJECT_ID":
    aiplatform.init(project=PROJECT_ID, location=REGION)
    print("✅ Vertex AI SDK Initialized.")
else:
    print("❌ Please configure PROJECT_ID first")

In [ ]:
# Get Endpoint Object
# Look up the endpoint by its display name
if PROJECT_ID and PROJECT_ID != "YOUR_PROJECT_ID":
    try:
        endpoints = aiplatform.Endpoint.list(filter=f'display_name="{ENDPOINT_DISPLAY_NAME}"')

        if endpoints:
            endpoint = endpoints[0]
            print(f"✅ Found Endpoint: {endpoint.resource_name}")
        else:
            print(f"❌ Endpoint '{ENDPOINT_DISPLAY_NAME}' not found in region {REGION}.")
            print("Please check the name and region.")
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Make sure you're authenticated: gcloud auth application-default login")
else:
    print("❌ Please configure PROJECT_ID first")

## Prepare Test Data and Get Predictions


In [ ]:
# Example reviews (Positive and Negative)
test_reviews = [
    "The product was fantastic and arrived early!",  # Positive
    "The product was terrible, broken, and arrived late."  # Negative
]

# Load vectorizer and transform text
try:
    vectorizer = joblib.load("vectorizer.joblib")
    print("Step 1: Transforming text...")
    sparse_matrix = vectorizer.transform(test_reviews)

    print("Step 2: Converting to list...")
    # Vertex AI expects python lists, not numpy arrays
    prediction_instances = sparse_matrix.toarray().tolist()

    print(f"✅ Data prepared. {len(prediction_instances)} instances ready.")

except FileNotFoundError:
    print("❌ 'vectorizer.joblib' not found. Please train and save the model first.")
    prediction_instances = None

In [ ]:
# Get Prediction
if 'endpoint' in locals() and prediction_instances:
    print("Sending prediction request...")
    try:
        response = endpoint.predict(instances=prediction_instances)

        print("\n--- Prediction Results ---")
        for i, prediction in enumerate(response.predictions):
            print(f"Review {i+1}: {test_reviews[i]}")
            print(f"Prediction: {prediction}")
            sentiment = "POSITIVE ✅" if prediction == 1 else "NEGATIVE ❌"
            print(f"Sentiment: {sentiment}\n")

    except Exception as e:
        print(f"❌ Error during prediction: {str(e)}")
else:
    print("❌ Endpoint not configured or data not prepared")

---

## 📝 Quick Reference

### Using this notebook:

1. Configure your GCP project (use config.py or direct configuration)
2. Train the model (or load existing joblib files)
3. Get endpoint reference
4. Make predictions

### For pipeline deployment (sending raw text):

See `07_Pipeline_Deployment/` and `08_Hit_Pipeline_Deployment/` notebooks.
